**Configuração, Inicialização e Carregamento de Dados**

# PREPARANDO O AMBIENTE

In [22]:
!pip install pyspark # Instala a biblioteca PySpark no ambiente de execução
from pyspark.sql import SparkSession # Importa a classe SparkSession, o ponto de entrada para o Spark
from pyspark.sql.functions import count, avg, max, min, variance, round, lit, last, first, desc, asc # Importa funções de agregação, formatação e ordenação a serem usadas nas análises
from pyspark.sql.window import Window # Importa a Classe Window, essencial para definir janelas de cálculo (como médias cumulativas)

In [23]:
# Cria ou obtém a SparkSession e define o nome da aplicação
spark = SparkSession.builder \
.appName("Agregacao_EDA") \
.getOrCreate()

print("SparkSession iniciada e funções de agregação importadas!") # Confirma o início da sessão
print(spark) # Exibe a instância da SparkSession

SparkSession iniciada e funções de agregação importadas!


# LEITURA DO DATAFRAME

In [24]:
df_video = spark.read.parquet("/content/videos-preparados.snappy.parquet") # Carrega o DataFrame de vídeos a partir de um arquivo Parquet pré-processado

print("DataFrame 'df_video' lido.") # Confirma a leitura
df_video.printSchema() # Exibe a estrutura (schema) do DataFrame
print(f"total de registros: {df_video.count()}") # Mostra a contagem total de registros

DataFrame 'df_video' lido.
root
 |-- Title: string (nullable = true)
 |-- Video ID: string (nullable = true)
 |-- Published At: date (nullable = true)
 |-- Keyword: string (nullable = true)
 |-- Likes: integer (nullable = true)
 |-- Comments: integer (nullable = true)
 |-- Views: integer (nullable = true)
 |-- Interaction: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Keyword Index: double (nullable = true)
 |-- Features PCA: vector (nullable = true)
 |-- Features Normal: vector (nullable = true)
 |-- Features: vector (nullable = true)

total de registros: 1869


**Agregações Básicas e Ordenação**

# CONTAGEM POR 'KEYWORD'

In [25]:
# Agrupa por Keyword e conta o número de registros (vídeos) em cada grupo. lit(1) é usado como valor constante para a função count
df_count_by_keyword = df_video.groupBy("Keyword").agg(
    count(lit(1)).alias("quantidade de Vídeos")
)

print("contagem de vídeos por palavra-chave:") # Mensagem de status
df_count_by_keyword.show(10) # Exibe as 10 primeiras linhas da contagem por palavra-chave

contagem de vídeos por palavra-chave:
+----------------+--------------------+
|         Keyword|quantidade de Vídeos|
+----------------+--------------------+
|computer science|                  48|
|            lofi|                  40|
|         finance|                  39|
|             cnn|                  50|
|           apple|                  42|
|            news|                  39|
|         mukbang|                  45|
|       education|                  24|
|       interview|                  50|
|          crypto|                  50|
+----------------+--------------------+
only showing top 10 rows



# MÉDIA DE 'INTERACTION' POR 'KEYWORD'

In [26]:
# Agrupa por Keyword e calcula a média da coluna Interaction
df_avg_interaction = df_video.groupBy("Keyword").agg(
    avg("Interaction").alias("Média de Interações")
)

print("Média de interações por palavra-chave:") # Mensagem de status
df_avg_interaction.show(10) # Exibe as 10 primeiras médias de interação por palavra-chave

Média de interações por palavra-chave:
+----------------+--------------------+
|         Keyword| Média de Interações|
+----------------+--------------------+
|computer science|  1226793.0208333333|
|            lofi|         4167085.875|
|         finance|   708542.9487179487|
|             cnn|           570650.86|
|           apple|1.0873628214285715E7|
|            news|  251688.71794871794|
|         mukbang|1.1053630377777778E7|
|       education|         2750838.625|
|       interview|          3044867.04|
|          crypto|            413676.2|
+----------------+--------------------+
only showing top 10 rows



# RANK POR MÁXIMO DE 'INTERACTION'

In [27]:
# 1. Agregação: Calcule o valor máximo de 'Interaction'
# Agrupa por Keyword e calcula o valor máximo da coluna Interaction
df_max_interaction = df_video.groupBy("Keyword").agg(
    max("Interaction").alias("Rank Interactions")
)

In [28]:
# 2. Ordenação: Ordene pela nova coluna em ordem decrescente (do maior para o menor)
df_ranked_interaction = df_max_interaction.orderBy(desc("Rank Interactions")) # Ordena o resultado anterior em ordem decrescente (desc), mostrando as palavras-chave com maior interação máxima primeiro

print("Máxima Interação por palavra-chave, ordenado (Rank):") # Mensagem de status
df_ranked_interaction.show(10) # Exibe o Top 10 de palavras-chave por máxima interação

Máxima Interação por palavra-chave, ordenado (Rank):
+--------+-----------------+
| Keyword|Rank Interactions|
+--------+-----------------+
| animals|       1593623628|
|   music|        922551152|
|     bed|        532691631|
| history|        440187490|
|   apple|        429916936|
| mrbeast|        300397699|
|  google|        239385460|
|business|        210025196|
|   cubes|        170925917|
|  sports|        106924567|
+--------+-----------------+
only showing top 10 rows



**Análise Estatística Detalhada e Datas**

# MÉDIA E VARIÂNCIA DE 'VIEWS'

In [29]:
# Agrupa por Keyword e calcula a média e a variância da coluna Views
df_view_stats = df_video.groupBy("Keyword").agg(
    avg("Views").alias("Média de Views"),
    variance("Views").alias("Variância de Views")
)

print("Média e variância de views por palavra-chave:") # Mensagem de status
df_view_stats.show(10) # Exibe as estatísticas de views

Média e variância de views por palavra-chave:
+----------------+--------------------+--------------------+
|         Keyword|      Média de Views|  Variância de Views|
+----------------+--------------------+--------------------+
|computer science|  1191958.7083333333| 2.81219868165102E12|
|            lofi|           4089363.0|1.846209641476677...|
|         finance|   694223.4358974359|3.304483175097042...|
|             cnn|           554240.38|1.563423618468118...|
|           apple|1.0746930452380951E7|4.299927977442589E15|
|            news|   247492.1794871795|1.067512576672564...|
|         mukbang|1.0904772355555555E7|5.586073238973179...|
|       education|  2684432.8333333335|1.833572249339214...|
|       interview|          2966111.86|1.819220996034335E13|
|          crypto|           404608.22|3.513691634369074E12|
+----------------+--------------------+--------------------+
only showing top 10 rows



# ESTATÍSTICAS DE 'VIEWS' SEM DECIMAIS

In [30]:
# Agrupa por Keyword, calcula a média, mínimo e máximo. A média é arredondada para 0 casas decimais (round(..., 0))
df_views_summary = df_video.groupBy("Keyword").agg(
    round(avg("Views"), 0).alias("Média de views (int)"),
    min("Views").alias("Mínimo de Views"),
    max("Views").alias("Máximo de Views")
)

print("Estatísticas de views por palavra-chave (média arredondada):") # Mensagem de status
df_views_summary.show(10) # Exibe as estatísticas com a média arredondada

Estatísticas de views por palavra-chave (média arredondada):
+----------------+--------------------+---------------+---------------+
|         Keyword|Média de views (int)|Mínimo de Views|Máximo de Views|
+----------------+--------------------+---------------+---------------+
|computer science|           1191959.0|          16115|        7004107|
|            lofi|           4089363.0|           6817|       84747957|
|         finance|            694223.0|           1195|        9450554|
|             cnn|            554240.0|          51269|        1889320|
|           apple|          1.074693E7|           1954|      425478119|
|            news|            247492.0|          10642|        1465011|
|         mukbang|         1.0904772E7|           3618|       86169225|
|       education|           2684433.0|           6611|       17103736|
|       interview|           2966112.0|           2587|       22529756|
|          crypto|            404608.0|           1599|       11805668|
+--

# PRIMEIRO E ÚLTIMO 'PUBLISHED AT'

In [31]:
# Agrupa por Keyword e encontra a data de publicação mínima (min) e máxima (max), definindo o período de publicação por palavra-chave
df_date_range = df_video.groupBy("Keyword").agg(
    min("Published At").alias("Primeira Publicação"),
    max("Published At").alias("Última Publicação")
)

print("Período de publicação por palavra-chave:") # Mensagem de status.
df_date_range.show(10) # Exibe o período de publicação

Período de publicação por palavra-chave:
+----------------+-------------------+-----------------+
|         Keyword|Primeira Publicação|Última Publicação|
+----------------+-------------------+-----------------+
|computer science|         2009-08-20|       2022-08-12|
|            lofi|         2019-12-08|       2022-08-24|
|         finance|         2012-11-27|       2022-08-24|
|             cnn|         2022-07-14|       2022-08-24|
|           apple|         2016-11-02|       2022-08-24|
|            news|         2022-08-18|       2022-08-24|
|         mukbang|         2020-02-29|       2022-08-24|
|       education|         2008-07-25|       2022-08-24|
|       interview|         2016-01-05|       2022-08-24|
|          crypto|         2022-03-11|       2022-08-24|
+----------------+-------------------+-----------------+
only showing top 10 rows



**Contagem Distinta e Tendência Temporal**

# CONTAGEM DE TÍTULOS (NORMAL vs. ÚNICA)

In [32]:
from pyspark.sql.functions import countDistinct, count

# Usa count para obter o total de títulos e countDistinct para obter o total de títulos únicos no DataFrame inteiro
df_title_counts = df_video.agg(
    count("Title").alias("Contagem Total de Títulos"),
    countDistinct("Title").alias("Contagem única de Títulos")
)

# Se os números forem diferentes, significa que há títulos duplicados no DataFrame.
df_title_counts.show() # Exibe as contagens para comparar se há títulos duplicados

+-------------------------+-------------------------+
|Contagem Total de Títulos|Contagem única de Títulos|
+-------------------------+-------------------------+
|                     1869|                     1854|
+-------------------------+-------------------------+



# CONTAGEM POR ANO (ASCENDENTE) E ANO/MÊS (ASCENDENTE)

In [33]:
# 1 Contagem por Ano
# Agrupa por Year, conta os vídeos e ordena o resultado por ano de forma ascendente
df_count_by_year = df_video.groupBy("Year").agg(
    count(lit(1)).alias("Vídeos por ano")
).orderBy(asc("Year")) # Ordena o ano de forma ascendente

print("Contagem de vídeos por ano (Ascendente):") # Mensagem de status.
df_count_by_year.show() # Exibe a tendência anual de volume de vídeos

Contagem de vídeos por ano (Ascendente):
+----+--------------+
|Year|Vídeos por ano|
+----+--------------+
|2007|             2|
|2008|             1|
|2009|             9|
|2010|             6|
|2011|             4|
|2012|            12|
|2013|             6|
|2014|            10|
|2015|            15|
|2016|            34|
|2017|            47|
|2018|            57|
|2019|            86|
|2020|           158|
|2021|           229|
|2022|          1193|
+----+--------------+



In [34]:
# 2 Contagem por Ano e Mês
# Agrupa por Year e Month, conta os vídeos e ordena o resultado por ano e, em seguida, por mês (ambos ascendentes)
df_count_by_year_month = df_video.groupBy("Year", "Month").agg(
    count(lit(1)).alias("Videos por ano/mês")
).orderBy(asc("Year"), asc("Month")) # Ordena pelo Ano, depois pelo Mês

print("contagem de vídeos por ano e mês (ascendente):") # Mensagem de status
df_count_by_year_month.show(10) # Exibe a tendência mensal/anual de volume de vídeos

contagem de vídeos por ano e mês (ascendente):
+----+-----+------------------+
|Year|Month|Videos por ano/mês|
+----+-----+------------------+
|2007|    7|                 1|
|2007|   12|                 1|
|2008|    7|                 1|
|2009|    2|                 2|
|2009|    6|                 2|
|2009|    7|                 1|
|2009|    8|                 1|
|2009|   10|                 1|
|2009|   12|                 2|
|2010|    3|                 1|
+----+-----+------------------+
only showing top 10 rows



**Média Acumulativa (Window Function)**

# MÉDIA ACUMULATIVA DE 'LIKES'

In [35]:
# 1. Definição da Janela:
# Define uma janela que: 1) Particiona por Keyword (cálculo independente por palavra-chave); 2) Ordena por Year (define a sequência temporal); 3) Inclui todos os registros desde o início da partição (unboundedPreceding) até o registro atual (currentRow)
window_spec = Window.partitionBy("Keyword").orderBy("Year").rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [36]:
# 2. Cálculo das Estatísticas por Ano/Keyword:
# Calcula a média de Likes por Year e Keyword, criando uma base para o cálculo cumulativo
df_annual_stats = df_video.groupBy("Year", "Keyword").agg(
    avg("Likes").alias("média anual de likes")
).orderBy("Keyword", "Year")

In [37]:
from pyspark.sql.functions import col

# 3. Aplicar a Média Acumulativa (Avg do Likes Anual sobre a Janela)
# Aplica a função de janela (avg(...) sobre window_spec) ao média anual de likes. Para cada ano, ele calcula a média de todas as médias anuais anteriores daquela Keyword
df_cumulativo_avg = df_annual_stats.withColumn(
    "Média acumulativa de likes",
    avg(col("Média anual de likes")).over(window_spec)
)

print("Média anual e média acumulativa de likes por keyword: ") # Mensagem de status
df_cumulativo_avg.show(20) # Exibe o resultado do cálculo cumulativo

Média anual e média acumulativa de likes por keyword: 
+----+-------+--------------------+--------------------------+
|Year|Keyword|média anual de likes|Média acumulativa de likes|
+----+-------+--------------------+--------------------------+
|2009|animals|           1357197.0|                 1357197.0|
|2010|animals|            203367.0|                  780282.0|
|2013|animals|         1.1025176E7|         4195246.666666667|
|2014|animals|           3381630.0|                 3991842.5|
|2019|animals|           1103713.0|                 3414216.6|
|2020|animals|   769652.1111111111|         2973455.851851852|
|2021|animals|           112729.75|        2564780.6944444445|
|2022|animals|  30335.214285714286|         2247975.009424603|
|2016|  apple|           4144389.0|                 4144389.0|
|2021|  apple|             38261.0|                 2091325.0|
|2022|  apple|             19416.6|        1400688.8666666665|
|2020|   asmr|            148120.0|                  148120.0|


**Encerramento da Sessão**

# PARANDO A SPARK SESSION

In [38]:
spark.stop() # Encerra a SparkSession e libera os recursos alocados
print("SparkSession encerrada.") # Mensagem de conclusão

SparkSession encerrada.
